# 本文件说明
- 数据库里导出数据，本地模型、线上模型测试

# 基本设置

In [1]:
import numpy as np
import pandas as pd

import os

import requests,json
from sklearn.externals import joblib

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
from toolkits.setup.date_time import get_day_list
from toolkits.setup import specific_func

from toolkits.nlp import pre_cor_circ
from toolkits.nlp import pre_cor_cbrc

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.018 seconds.
Prefix dict has been built succesfully.


## 一些函数

In [3]:
def get_server_res_yjh(data, url, col_name):
    '''
    服务器接口测试程序
    传入 dict, 传出 DataFrame
    '''
    # data = {'record':[{'id':0,'title':'ss','content':'zzz'},]}
    # data = {"record":marked_human_data.iloc[:5,:3].to_dict(orient = 'records')}
    # url "http://47.93.77.19:10000/correlation_negative"
    headers={'content-type':'application/json'}
    result = requests.post(url,
                      data = json.dumps(data),
                      headers=headers, allow_redirects=True)
    # print(result.text)
    json_data = json.loads(result.text)
    parse_data = []
#     elapsed_time = json_data['elapsed_time']
    for i in range(len(json_data['docs'])):
        parse_data.append([json_data['docs'][i]['id'],
                          json_data['docs'][i][col_name]])
    parse_data = pd.DataFrame(parse_data, columns = ['id', col_name])    
    return parse_data #, elapsed_time

In [4]:
def get_serve_data_yjh(day_list, sql_one_day, url, col_name, save_filename):    
    chunksize = 1000
    for day_select in day_list:
        print('-- day_select: ', day_select)
        mysql_data = pd.read_sql(eval(sql_one_day), engine, chunksize= chunksize)
        num = 1
        combined_data = pd.DataFrame()
        for tmp_data in mysql_data:  
            print('---- loop num: ', num, 'tmp_data: ', tmp_data.shape)
            data = {"record":tmp_data.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
            parse_data = get_server_res_yjh(data, url, col_name)

            parse_data.columns = ['id', 'predict_label']
            
            parse_data['label'] = ''
            combined_tmp = pd.merge(parse_data, tmp_data, on = 'id', how = 'inner')
            combined_data = pd.concat([combined_tmp, combined_data])

        combined_data['predict_label'] = combined_data['predict_label'].apply(lambda x:class_name_dict[x])
        combined_data['group_id'] = combined_data['group_id'].apply(lambda x:group_dict[str(x)])
        combined_data.to_excel(eval(save_filename), index = False)
        print(combined_data['predict_label'].value_counts())

In [5]:
def get_server_res(data, url, col_name):
    '''
    服务器接口测试程序
    传入 dict, 传出 DataFrame
    '''
    # data = {'record':[{'id':0,'title':'ss','content':'zzz'},]}
    # data = {"record":marked_human_data.iloc[:5,:3].to_dict(orient = 'records')}
    # url "http://47.93.77.19:10000/correlation_negative"
    headers={'content-type':'application/json'}
    result = requests.post(url,
                      data = json.dumps(data),
                      headers=headers, allow_redirects=True)
    # print(result.text)
    json_data = json.loads(result.text)
    parse_data = []
    elapsed_time = json_data['elapsed_time']
    for i in range(len(json_data['docs'])):
        parse_data.append([json_data['docs'][i]['id'],
                          json_data['docs'][i][col_name]])
    parse_data = pd.DataFrame(parse_data, columns = ['id', col_name])    
    return parse_data, elapsed_time

In [6]:
def get_serve_data(day_list, sql_one_day, url, col_name):
    combined_data = pd.DataFrame()
    for day_select in day_list:
        print('-- day_select: ', day_select)
        mysql_data = pd.read_sql(eval(sql_one_day), engine)
        print('去空值前：', mysql_data.shape)
        mysql_data = mysql_data.drop_duplicates(subset = ['title', 'content'])
        print('去空值后：', mysql_data.shape)
        data = {"record":mysql_data.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
        
        parse_data, elapsed_time = get_server_res(data, url)
        print('elapsed_time: ', elapsed_time)
        
        parse_data.columns = ['id', 'predict_label']
        parse_data['predict_label'] = parse_data['predict_label'].apply(lambda x:class_name_dict[x])
        parse_data['label'] = ''
        combined_cor = pd.merge(parse_data, mysql_data, on = 'id', how = 'inner')
        combined_data = pd.concat([combined_data, combined_cor], axis = 0)

        print(combined_cor['predict_label'].value_counts())
    return combined_data

# 基本信息

In [7]:
label_dic={'补录':0,'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
class_name_dict = {v: k for k, v in label_dic.items()}
class_name_dict

{0: '补录',
 1: '监管',
 2: '行业',
 3: '产品销售',
 4: '资本市场',
 5: '公司内部管理',
 6: '消费服务',
 7: '其他相关报道',
 8: '噪音'}

In [8]:
group = '1-新闻，2-论坛，3-博客，4-微博，5-纸媒，6-视频，7-外媒，8-广播，9-电视，11-微信，13-新闻客户端，15-推特'
group_dict = dict([x.split('-') for x in group.split('，')])
group_dict

{'1': '新闻',
 '11': '微信',
 '13': '新闻客户端',
 '15': '推特',
 '2': '论坛',
 '3': '博客',
 '4': '微博',
 '5': '纸媒',
 '6': '视频',
 '7': '外媒',
 '8': '广播',
 '9': '电视'}

# 保险业--旧

In [9]:
engine = specific_func.get_engine('circ')

## mysql 数据

In [17]:
# day_select = '2018-09-09'
day_list = get_day_list('2018-10-17', '2018-10-19')
print(day_list)

['2018-10-18', '2018-10-19']


### 八分类

#### 获取数据

In [16]:
for day_select in day_list:
    print('-- day_select: ', day_select)

    # 相关数据
    sql_one_day = "select t1.id, t1.group_id,t1.classify as predict_label,\
                        t1.title,t2.center as content, t1.publishtime as publishtime \
                        from wise_web_docinfo t1, wise_web_docinfo_center t2 \
                            where t1.id=t2.doc_id \
                                  and  date_format(t1.publishtime, '%%Y-%%m-%%d') = '{0}' \
                                  group by t1.titlehash".format(day_select) # 
    # # titlehash 去重后
    circ_cor = pd.read_sql(sql_one_day, engine)
    print('circ_cor: ', circ_cor.shape  )
    
    # 不相关数据
    sql_one_day = "select t1.id, t1.group_id,t1.title,t2.center as content, t1.publishtime as publishtime \
                        from wise_web_docinfo_uncorr t1, wise_web_docinfo_center_uncurr t2 \
                            where t1.id=t2.doc_id \
                                  and t1.publishtime >= '{0} 8:00:00' \
                              and t1.publishtime <= '{0} 14:00:00'".format(day_select)
    # 一段时间
    circ_uncor = pd.read_sql(sql_one_day, engine)
    circ_uncor.insert(2, 'predict_label', 8) # 噪音
    print('circ_uncor: ', circ_uncor.shape)

    circ_data = pd.concat([circ_cor, circ_uncor], axis = 0)
    print('去重前：', circ_data.shape)
    circ_data = circ_data.drop_duplicates(subset = 'title')
    print('去重后：', circ_data.shape)  
    circ_data = circ_data.dropna(subset = ['content'], axis = 0)
    print('去空值后：', circ_data.shape)  

    circ_data['predict_label'] = circ_data['predict_label'].apply(lambda x:class_name_dict[x])
    circ_data['group_id'] = circ_data['group_id'].apply(lambda x:group_dict[str(x)])
    circ_data.insert(3, 'label', '')
    fea_filename = 'circ_result_class/result/circ_class_predict_mysql_%s.xlsx'%day_select
    circ_data.to_excel(fea_filename, index = False)
    print(circ_data.shape)
    print(circ_data['predict_label'].value_counts())

-- day_select:  2018-10-18
circ_cor:  (2886, 6)
circ_uncor:  (3506, 6)
去重前： (6392, 6)
去重后： (5306, 6)
去空值后： (5306, 6)
(5306, 7)
噪音        2420
资本市场       759
监管         549
公司内部管理     387
行业         327
消费服务       318
其他相关报道     310
产品销售       232
补录           4
Name: predict_label, dtype: int64


#### 合并 & 保存

In [18]:
combined_data = pd.DataFrame()
for day_select in day_list:
    tmp_data = pd.read_excel('circ_result_class/result/circ_class_predict_mysql_%s.xlsx'%day_select)
    combined_data = pd.concat([combined_data, tmp_data], axis = 0)

combined_data = combined_data[combined_data['predict_label'] != '补录']
print(combined_data.shape)  
print('去重前：', combined_data.shape)
combined_data = combined_data.drop_duplicates(subset = 'title')
print('去重后：', combined_data.shape)  
combined_data = combined_data.dropna(subset = ['content'], axis = 0)
print('去空值后：', combined_data.shape)  

print(combined_data['predict_label'].value_counts())
combined_data.head()

(8039, 7)
去重前： (8039, 7)
去重后： (6669, 7)
去空值后： (6668, 7)
噪音        3714
资本市场       783
监管         569
公司内部管理     395
行业         336
消费服务       324
其他相关报道     313
产品销售       234
Name: predict_label, dtype: int64


,id,group_id,predict_label,label,title,content,publishtime
0,11434656,新闻,监管,NaN,银保监局整合加速：地方监管36名筹备组负责人已确定,独家丨最全！银保监局整合加速：36名筹备组负责人名单 21世纪经济报道 21财经APP...,2018-10-18 22:06:00
1,11434782,新闻,资本市场,NaN,中国人寿(601628)融资融券信息(10-18),中国人寿(601628)融资融券信息(10-18)2018年10月19日 07:39来源：东...,2018-10-19 07:39:13
2,11432632,新闻,产品销售,NaN,相互保2天吸引超150万人参与，专家预计每年费用不会超过一两百,(图片) 近日，蚂蚁保险联合信美相互推出了一种互助型健康保障服务“相互保”，在他人生病时参...,2018-10-19 00:00:00
3,11425553,微博,公司内部管理,NaN,【银保监会合并后首张保险业罚单出炉！新华保险回应：严肃整改 引以为戒】银保监会今日下发监管合...,【银保监会合并后首张保险业罚单出炉！新华保险回应：严肃整改 引以为戒】银保监会今日下发监管合...,2018-10-18 18:14:19
4,11425801,新闻,资本市场,NaN,今日股市收评：短线变盘行情或一触即发,0,2018-10-18 18:15:00


In [21]:
fea_filename = 'circ_result_class/result/circ_class_predict_mysql_20181019(1018-1019).xlsx'
# sel_col = ['行业','资本市场', '消费服务', '公司内部管理', '监管']
# sel_col = ['其他相关报道','行业',  '公司内部管理', '监管']
sel_col = combined_data['predict_label'].unique().tolist()
sel_data = combined_data[combined_data['predict_label'].isin(sel_col)]
print(sel_data['predict_label'].value_counts())
with pd.ExcelWriter(fea_filename) as writer:
    for label in sel_data['predict_label'].unique():
        tmp_data = sel_data[sel_data['predict_label'] == label]
        if tmp_data.shape[0] > 200:
            N = 200
        else :
            N = tmp_data.shape[0]
        if label == '公司内部管理': 
            N = 200
        tmp_data.sample(n = N, axis = 0, random_state=3).to_excel(writer,label, index = False)
    
    writer.save()

噪音        3714
资本市场       783
监管         569
公司内部管理     395
行业         336
消费服务       324
其他相关报道     313
产品销售       234
Name: predict_label, dtype: int64


## 本地模型

### 八分类

In [19]:
from sklearn.externals import joblib
pipeline_old = joblib.load( "model/circ_8classifier_1015.pkl.z")

In [24]:
combined_data['title_content'] = combined_data['title'].astype(str) + '。' + combined_data['content'].astype(str)
title_content = pre_cor_circ.handle_contents(combined_data['title_content'].tolist())
print(len(title_content))

8511


In [ ]:
local_label = pipeline_old.predict(title_content)
local_proba = pipeline_old.predict_proba(title_content)

combined_data['local_label'] = local_label
combined_data['local_proba'] = local_proba.max(axis = 1)
combined_data['local_label'] = combined_data['local_label'].apply(lambda x:class_name_dict[x])
print(combined_data.shape)
combined_data.iloc[:2, :]

#### 线上线下一致性: mysql 与 local

In [34]:
combined_data['R_W'] = combined_data.apply(lambda x: 'Right' if x['local_label'] == x['predict_label'] else 'Wrong', axis = 1)
print(combined_data[combined_data['R_W'] == 'Right'].shape[0]/combined_data.shape[0])
print(combined_data['R_W'].value_counts())
combined_data[combined_data['R_W'] == 'Wrong'].pivot_table(index = ['local_label'], columns = ['predict_label'], 
                                                            aggfunc = [len], values = ['id'], 
                                                            fill_value = 0, margins = True)

0.9836681941017507
Right    8372
Wrong     139
Name: R_W, dtype: int64


len                                       
                id                                       
predict_label 产品销售 公司内部管理 其他相关报道  噪音 消费服务 监管 行业 资本市场  All
local_label                                              
产品销售             0      1      1   2    0  0  0    0    4
公司内部管理           1      0      3   6    0  0  0    0   10
其他相关报道           0      4      0   6    0  0  1    0   11
噪音              17      2      2   0    1  2  4   12   40
消费服务             0      1      1   9    0  0  1    1   13
监管               0      0      0  17    3  0  2    0   22
行业               2      2      0   5    4  1  0    0   14
资本市场             0      0      0  25    0  0  0    0   25
All             20     10      7  70    8  3  8   13  139

#### 线上线下一致性: online 与 local

In [ ]:
combined_data['id'] = range(combined_data.shape[0])
combined_data['title'] = combined_data['title'].astype(str) 
combined_data['content'] = combined_data['content'].astype(str)
data = {"record":combined_data.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
url = "http://47.93.77.19:10000/judge_correlation_i"
col_name = 'cor'
parse_data, elapsed_time = get_server_res(data, url, col_name)
parse_data.columns = ['id', 'online_label']
parse_data.head()

In [ ]:
combined_data = pd.merge(combined_data, parse_data, on  = 'id')
print(combined_data.shape)
combined_data['O_R_W'] = combined_data.apply(lambda x: 'Right' if x['local_label'] == x['online_label'] else 'Wrong', axis = 1)
print(combined_data[combined_data['O_R_W'] == 'Right'].shape[0]/combined_data.shape[0])
print(combined_data['O_R_W'].value_counts())
combined_data[combined_data['O_R_W'] == 'Wrong'].pivot_table(index = ['local_label'], columns = ['online_label'], 
                                                            aggfunc = [len], values = ['urlhash'], 
                                                            fill_value = 0, margins = True)

## 线上模型

### 八分类

# 银行业--旧

In [9]:
engine = specific_func.get_engine('cbrc')

In [10]:
# day_select = '2018-09-09'
day_list = get_day_list('2018-10-23', '2018-10-29')
print(day_list)

['2018-10-24', '2018-10-25', '2018-10-26', '2018-10-27', '2018-10-28', '2018-10-29']


## mysql 数据

### 八分类数据

#### 获取数据

In [59]:
# sql_circ_cor_one_day = "select t1.id, t1.publishtime, t1.title,t2.text as content \
#                             from elint_web_docinfo t1, wise_web_docinfo_text t2 \
#                                 where t1.id = t2.doc_id \
#                                   and date_format(t1.publishtime, '%%Y-%%m-%%d') = '{0}'".format('2018-08-07')
# # 实际
# circ_cor = pd.read_sql(sql_circ_cor_one_day, engine)
# print(circ_cor.shape)
# circ_cor.head()

In [11]:
for day_select in day_list:
    print('-- day_select: ', day_select)
    
    # 获取八分类
    sql_one_day = "select t2.urlhash, t1.traffic_id, t2.title as title_1\
                        from wise_web_classify_traffic_docinfo t1, wise_web_docinfo_basic t2 \
                            where t1.base_id=t2.id \
                                  and date_format(t2.publishtime, '%%Y-%%m-%%d') = '{0}' ".format(day_select)
    cbrc_flag = pd.read_sql(sql_one_day, engine)
    print('cbrc_flag：', cbrc_flag.shape)
    
    # 相关数据
    sql_one_day = "select t1.urlhash, t1.title,t2.text as content, t1.group_id, t1.publishtime as publishtime \
                        from elint_web_docinfo t1, wise_web_docinfo_text t2 \
                            where t1.id=t2.doc_id \
                                  and t1.publishtime >= '{0} 08:00:00' \
                                  and t1.publishtime <= '{0} 14:00:00' \
                                group by t1.titlehash".format(day_select)
    # titlehash 去重后
    cbrc_cor = pd.read_sql(sql_one_day, engine) 
    print('cbrc_cor：', cbrc_cor.shape)
    
    # 不相关数据
    sql_cbrc_uncor = "select urlhash, title, content, group_id, publishtime \
                            from wise_web_docinfo_uncor \
                            where date_format(publishtime, '%%Y-%%m-%%d') = '{0}'".format(day_select)
    cbrc_uncor = pd.read_sql(sql_cbrc_uncor, engine)  
    print('cbrc_uncor：', cbrc_uncor.shape)

    cbrc_data = pd.concat([cbrc_cor, cbrc_uncor], axis = 0)
    print('去重前：', cbrc_data.shape)
    cbrc_data = cbrc_data.drop_duplicates(subset = 'title')
    print('去重后：', cbrc_data.shape)  
    cbrc_data = cbrc_data.dropna(subset = ['content'], axis = 0)
    print('去空值后：', cbrc_data.shape)  

    cbrc_combined = pd.merge(cbrc_flag, cbrc_data, how = 'inner', on = 'urlhash')
    cbrc_combined['predict_label'] = cbrc_combined['traffic_id'].apply(lambda x:class_name_dict[x])
    cbrc_combined['group_id'] = cbrc_combined['group_id'].apply(lambda x:group_dict[str(x)])
    cbrc_combined['label'] = ''
    cbrc_combined = cbrc_combined[['urlhash', 'predict_label', 'label', 'title', 'content', 'group_id', 'publishtime']]
    fea_filename = 'cbrc_result_class/result/cbrc_class_predict_mysql_%s.xlsx'%day_select
    cbrc_combined.to_excel(fea_filename, index = False)
    print(cbrc_combined.shape)
    print(cbrc_combined['predict_label'].value_counts())
    

-- day_select:  2018-10-24
cbrc_flag： (204529, 3)
cbrc_cor： (6078, 5)
cbrc_uncor： (5005, 5)
去重前： (11083, 5)
去重后： (10044, 5)
去空值后： (10044, 5)
(9980, 7)
噪音        2870
消费服务      2151
行业        1660
监管        1604
资本市场       793
公司内部管理     655
其他相关报道     179
产品销售        68
Name: predict_label, dtype: int64
-- day_select:  2018-10-25
cbrc_flag： (231192, 3)
cbrc_cor： (6514, 5)
cbrc_uncor： (5006, 5)
去重前： (11520, 5)
去重后： (10383, 5)
去空值后： (10383, 5)
(10305, 7)
噪音        2769
消费服务      2232
行业        1795
监管        1795
资本市场       812
公司内部管理     683
其他相关报道     148
产品销售        71
Name: predict_label, dtype: int64
-- day_select:  2018-10-26
cbrc_flag： (194749, 3)
cbrc_cor： (5934, 5)
cbrc_uncor： (5004, 5)
去重前： (10938, 5)
去重后： (10057, 5)
去空值后： (10057, 5)
(9984, 7)
噪音        2845
消费服务      2061
监管        1662
行业        1612
资本市场       941
公司内部管理     597
其他相关报道     175
产品销售        91
Name: predict_label, dtype: int64
-- day_select:  2018-10-27
cbrc_flag： (105576, 3)
cbrc_cor： (2498, 5)
cbrc_uncor： (5

#### 合并 & 保存

In [12]:
combined_data = pd.DataFrame()
for day_select in day_list:
    tmp_data = pd.read_excel('cbrc_result_class/result/cbrc_class_predict_mysql_%s.xlsx'%day_select)
    combined_data = pd.concat([combined_data, tmp_data], axis = 0)

combined_data = combined_data[combined_data['predict_label'] != '补录']
print(combined_data.shape)  
print('去重前：', combined_data.shape)
combined_data = combined_data.drop_duplicates(subset = 'title')
print('去重后：', combined_data.shape)  
combined_data = combined_data.dropna(subset = ['content'], axis = 0)
print('去空值后：', combined_data.shape)  

print(combined_data['predict_label'].value_counts())
combined_data.iloc[:2, :]

(50168, 7)
去重前： (50168, 7)
去重后： (47595, 7)
去空值后： (47071, 7)
噪音        16612
消费服务       9039
行业         6842
监管         6470
资本市场       4331
公司内部管理     2450
其他相关报道      885
产品销售        442
Name: predict_label, dtype: int64


,urlhash,predict_label,label,title,content,group_id,publishtime
0,2994115534462627840,噪音,NaN,豪华中型车二十万出头 开上豪车不是梦,凤凰网汽车 长春 导购 ： 很多富裕的平民家庭在用车方面已经开始不仅追求代步了，而乘坐的舒适...,新闻客户端,2018-10-24 07:50:00
1,4661064664638386176,行业,NaN,苏宁金融研究院发布《2018年3季度互联网金融行业报告》,首先，简单通过几组数据，反映截至2018年第三季度互联网金融行业的市场结构。 股权融资： 据...,新闻,2018-10-24 08:00:00


In [13]:
fea_filename = 'cbrc_result_class/result/cbrc_class_predict_mysql_20181030(1024-1029).xlsx'
# sel_col = ['噪音', '消费服务', '公司内部管理', '监管',
#            '行业', '资本市场', '其他相关报道','产品销售']
# sel_col = ['消费服务', '公司内部管理', '监管', '行业', '产品销售']
sel_col = combined_data['predict_label'].unique().tolist()
sel_data = combined_data[combined_data['predict_label'].isin(sel_col)]
print(sel_data['predict_label'].value_counts())
with pd.ExcelWriter(fea_filename) as writer:
    for label in sel_data['predict_label'].unique():
        tmp_data = sel_data[sel_data['predict_label'] == label]
        if tmp_data.shape[0] > 300:
            N = 300
        else :
            N = tmp_data.shape[0]
        tmp_data.sample(n = N, axis = 0, random_state=42).to_excel(writer,label, index = False)
    
    writer.save()

噪音        16612
消费服务       9039
行业         6842
监管         6470
资本市场       4331
公司内部管理     2450
其他相关报道      885
产品销售        442
Name: predict_label, dtype: int64


## 本地模型

### 八分类

In [63]:
from sklearn.externals import joblib
pipeline_old = joblib.load( "model/cbrc_8classifier_1015.pkl.z")

In [64]:
combined_data['title_content'] = combined_data['title'].astype(str) + '。' + combined_data['content'].astype(str)
title_content = pre_cor_cbrc.handle_contents(combined_data['title_content'].tolist())
print(len(title_content))

local_label = pipeline_old.predict(title_content)
local_proba = pipeline_old.predict_proba(title_content)

25072


In [65]:
combined_data['local_label'] = local_label
combined_data['local_proba'] = local_proba.max(axis = 1)
combined_data['local_label'] = combined_data['local_label'].apply(lambda x:class_name_dict[x])
print(combined_data.shape)
combined_data.iloc[:2, :]

(25072, 10)


,urlhash,predict_label,label,title,content,group_id,publishtime,title_content,local_label,local_proba
0,4683329437982043136,公司内部管理,NaN,浦发银行： 拟成立金融科技直投基金,银行探索金融科技业务再现新模式。浦发银行日前宣布成立“科技合作共同体”，并与首批16家国内外...,纸媒,2018-09-25 00:02:17,浦发银行： 拟成立金融科技直投基金。银行探索金融科技业务再现新模式。浦发银行日前宣布成立“科...,公司内部管理,0.741746
1,4296445329237014016,监管,NaN,财经热点 Financial Topic,银保监会： 8月末制造业贷款余额17万亿□记者 钟源 北京报道 中国银行保险监督管理委员会新...,纸媒,2018-09-25 00:02:16,财经热点 Financial Topic。银保监会： 8月末制造业贷款余额17万亿□记者 钟...,监管,0.795061


#### 线上线下一致性: mysql 与 local

In [66]:
combined_data['R_W'] = combined_data.apply(lambda x: 'Right' if x['local_label'] == x['predict_label'] else 'Wrong', axis = 1)
print(combined_data[combined_data['R_W'] == 'Right'].shape[0]/combined_data.shape[0])
print(combined_data['R_W'].value_counts())
combined_data[combined_data['R_W'] == 'Wrong'].pivot_table(index = ['local_label'], columns = ['predict_label'], 
                                                            aggfunc = [len], values = ['urlhash'], 
                                                            fill_value = 0, margins = True)

0.691847479259732
Right    17346
Wrong     7726
Name: R_W, dtype: int64


len                                               
              urlhash                                               
predict_label    产品销售 公司内部管理 其他相关报道  噪音  消费服务    监管    行业 资本市场   All
local_label                                                         
产品销售                0     15      0   4  1042    35    89    0  1185
公司内部管理              0      0      2   0    45    57    58    1   163
其他相关报道              4     93      0  15   244   129    88    2   575
噪音                 19    161     29   0  1500  1332   790  113  3944
消费服务                9     25      1  16     0    37   282    2   372
监管                  1     15      1   7    60     0   117    1   202
行业                  7     56      0  16    77   280     0    2   438
资本市场                0     22      0  29   373   245   178    0   847
All                40    387     33  87  3341  2115  1602  121  7726

#### 线上线下一致性: online 与 local

In [67]:
combined_data['id'] = range(combined_data.shape[0])
combined_data['title'] = combined_data['title'].astype(str) 
combined_data['content'] = combined_data['content'].astype(str)

In [68]:
data = {"record":combined_data.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
url = "http://47.93.77.19:6001/judge_correlation_yjh"
col_name = 'sec'
parse_data = get_server_res_yjh(data, url, col_name)
parse_data.columns = ['id', 'online_label']
parse_data.head()

,id,online_label
0,0,5
1,1,1
2,2,1
3,3,1
4,4,8


In [69]:
combined_data = pd.merge(combined_data, parse_data, on  = 'id')
print(combined_data.shape)
combined_data['online_label'] = combined_data['online_label'].apply(lambda x:class_name_dict[x])
combined_data['O_R_W'] = combined_data.apply(lambda x: 'Right' if x['local_label'] == x['online_label'] else 'Wrong', axis = 1)
print(combined_data[combined_data['O_R_W'] == 'Right'].shape[0]/combined_data.shape[0])
print(combined_data['O_R_W'].value_counts())
combined_data[combined_data['O_R_W'] == 'Wrong'].pivot_table(index = ['local_label'], columns = ['online_label'], 
                                                            aggfunc = [len], values = ['urlhash'], 
                                                            fill_value = 0, margins = True)

(25072, 13)
0.7174138481174218
Right    17987
Wrong     7085
Name: O_R_W, dtype: int64


len                                                
             urlhash                                                
online_label    产品销售 公司内部管理 其他相关报道   噪音  消费服务    监管    行业 资本市场   All
local_label                                                         
产品销售               0     14      0    5   780    30    89    0   918
公司内部管理             0      0      3    3    37    57    58    1   159
其他相关报道             4     93      0   19   228   128    88    2   562
噪音                23    158     35    0  1238  1317   768  119  3658
消费服务              27     24      2   22     0    36   276    3   390
监管                 1     14      1   10    43     0   116    1   186
行业                 8     55      1   18    62   278     0    2   424
资本市场              11     22      0   33   303   243   176    0   788
All               74    380     42  110  2691  2089  1571  128  7085

#### 线上线下一致性: online 与 mysql

In [70]:
combined_data['O_R_W'] = combined_data.apply(lambda x: 'Right' if x['predict_label'] == x['online_label'] else 'Wrong', axis = 1)
print(combined_data[combined_data['O_R_W'] == 'Right'].shape[0]/combined_data.shape[0])
print(combined_data['O_R_W'].value_counts())
combined_data[combined_data['O_R_W'] == 'Wrong'].pivot_table(index = ['predict_label'], columns = ['online_label'], 
                                                            aggfunc = [len], values = ['urlhash'], 
                                                            fill_value = 0, margins = True)

0.9673739629865986
Right    24254
Wrong      818
Name: O_R_W, dtype: int64


len                                          
              urlhash                                          
online_label     产品销售 公司内部管理 其他相关报道   噪音 消费服务  监管  行业 资本市场  All
predict_label                                                  
产品销售                0      0      0    0    1   0   0    0    1
公司内部管理              0      0      1    8    2   1   1    1   14
噪音                  0      0      0    0    7   0   0    1    8
消费服务              296      9     19  263    0  25  21   63  696
监管                  1      1      0   23    7   0  10    3   45
行业                  4      1      2   19   11   9   0    3   49
资本市场                1      0      0    4    0   0   0    0    5
All               302     11     22  317   28  35  32   71  818

## 线上模型

### 八分类数据

#### 计算数据

In [ ]:
# url = "http://192.168.0.104:6001/judge_correlation_yjh"
url = "http://47.93.77.19:6001/judge_correlation_yjh"

day_list = get_day_list('2018-09-08', '2018-09-10')
print(day_list)

In [ ]:
# 不相关
sql_cbrc_uncor = '''"select id, title, content, group_id, publishtime \
                        from wise_web_docinfo_uncor \
                        where date_format(publishtime, '%%Y-%%m-%%d') = '{0}'".format(day_select)'''

save_filename = ''''cbrc_result_class/result/cbrc_uncor_predict_%s.xlsx'%day_select'''
get_serve_data_yjh(day_list, sql_cbrc_uncor, url, 'sec', save_filename)

In [ ]:
# 相关
sql_cbrc_cor = '''
"select t1.id, t1.title,t2.text as content, t1.group_id, t1.publishtime as publishtime \
                    from elint_web_docinfo t1, wise_web_docinfo_text t2 \
                        where t1.id=t2.doc_id \
                              and date_format(t1.publishtime, '%%Y-%%m-%%d') = '{0}' \
                              group by t1.titlehash".format(day_select)
'''

save_filename = ''''cbrc_result_class/result/cbrc_cor_predict_%s.xlsx'%day_select'''
get_serve_data_yjh(day_list, sql_cbrc_cor, url, 'sec', save_filename)

#### 合并 & 保存

In [ ]:
combined_uncor = pd.DataFrame()
for day_select in day_list:
    tmp_data = pd.read_excel('cbrc_result_class/result/cbrc_uncor_predict_%s.xlsx'%day_select)
    combined_uncor = pd.concat([combined_uncor, tmp_data], axis = 0)
    
print(combined_uncor.shape)  
print('去重前：', combined_uncor.shape)
combined_uncor = combined_uncor.drop_duplicates(subset = ['title', 'content'])
print('去重后：', combined_uncor.shape)  
combined_uncor = combined_uncor.dropna(subset = ['title', 'content'], axis = 0)
print('去空值后：', combined_uncor.shape)  

print(combined_uncor['predict_label'].value_counts())
combined_uncor.head()

In [ ]:
fea_filename = 'cbrc_result_class/result/cbrc_uncor_predict_class_20180911(0909-0910).xlsx'
with pd.ExcelWriter(fea_filename) as writer:
    for label in combined_uncor['predict_label'].unique():
        tmp_data = combined_uncor[combined_uncor['predict_label'] == label]
        if tmp_data.shape[0] > 100:
            N = 200
            tmp_data.sample(n = N, axis = 0, random_state=42).to_excel(writer,label, index = False)
    
    writer.save()

In [ ]:
combined_cor = pd.DataFrame()
for day_select in day_list:
    tmp_data = pd.read_excel('cbrc_result_class/result/cbrc_cor_predict_%s.xlsx'%day_select)
    combined_cor = pd.concat([combined_cor, tmp_data], axis = 0)

print(combined_cor.shape)  
print('去重前：', combined_cor.shape)
combined_cor = combined_cor.drop_duplicates(subset = ['title', 'content'])
print('去重后：', combined_cor.shape)  
combined_cor = combined_cor.dropna(subset = ['title', 'content'], axis = 0)
print('去空值后：', combined_cor.shape)  

print(combined_cor.shape)  
print(combined_cor['predict_label'].value_counts())
combined_cor.head()

In [ ]:
fea_filename = 'cbrc_result_class/result/cbrc_cor_predict_class_20180911(0909-0910).xlsx'
with pd.ExcelWriter(fea_filename) as writer:
    for label in combined_cor['predict_label'].unique():
        tmp_data = combined_cor[combined_cor['predict_label'] == label]
        if tmp_data.shape[0] > 700:
            N = 200
            tmp_data.sample(n = N, axis = 0, random_state=42).to_excel(writer,label, index = False)
    
    writer.save()

# 银行业与保险业--新

In [ ]:
engine = specific_func.get_engine('cbirc')

## mysql 数据

## 本地模型

### 八分类

In [19]:
from sklearn.externals import joblib
pipeline_old = joblib.load( "model/circ_8classifier_1015.pkl.z")

In [24]:
combined_data['title_content'] = combined_data['title'].astype(str) + '。' + combined_data['content'].astype(str)
title_content = pre_cor_circ.handle_contents(combined_data['title_content'].tolist())
print(len(title_content))

8511


In [36]:
local_label = pipeline_old.predict(title_content)
local_proba = pipeline_old.predict_proba(title_content)

combined_data['local_label'] = local_label
combined_data['local_proba'] = local_proba.max(axis = 1)
combined_data['local_label'] = combined_data['local_label'].apply(lambda x:class_name_dict[x])
print(combined_data.shape)
combined_data.iloc[:2, :]

(8511, 12)


,id,group_id,predict_label,label,title,content,publishtime,title_content,local_label,predict_proba,R_W,local_proba
0,10910729,新闻,其他相关报道,NaN,阳光人寿天津分公司组织开展贫困生帮扶活动,近日，为深入贯彻党的十九大工作报告关于坚决打赢脱贫攻坚战的新任务、新要求，深入落实中央、...,2018-09-20 10:20:46,阳光人寿天津分公司组织开展贫困生帮扶活动。 近日，为深入贯彻党的十九大工作报告关于坚决打赢...,其他相关报道,0.995057,Right,0.995057
1,10908460,微信,监管,NaN,监管风暴再度来袭，3000个虚拟货币交易账户被关闭,链尚财经 链尙未来，链上你我 从去年的“9.4风暴”开始，监管层对数字货币的打击态度从未放松...,2018-09-20 08:23:52,监管风暴再度来袭，3000个虚拟货币交易账户被关闭。链尚财经 链尙未来，链上你我 从去年的“...,监管,0.781802,Right,0.781802


#### 线上线下一致性

In [34]:
combined_data['R_W'] = combined_data.apply(lambda x: 'Right' if x['local_label'] == x['predict_label'] else 'Wrong', axis = 1)
print(combined_data[combined_data['R_W'] == 'Right'].shape[0]/combined_data.shape[0])
print(combined_data['R_W'].value_counts())
combined_data[combined_data['R_W'] == 'Wrong'].pivot_table(index = ['local_label'], columns = ['predict_label'], 
                                                            aggfunc = [len], values = ['id'], 
                                                            fill_value = 0, margins = True)

0.9836681941017507
Right    8372
Wrong     139
Name: R_W, dtype: int64


len                                       
                id                                       
predict_label 产品销售 公司内部管理 其他相关报道  噪音 消费服务 监管 行业 资本市场  All
local_label                                              
产品销售             0      1      1   2    0  0  0    0    4
公司内部管理           1      0      3   6    0  0  0    0   10
其他相关报道           0      4      0   6    0  0  1    0   11
噪音              17      2      2   0    1  2  4   12   40
消费服务             0      1      1   9    0  0  1    1   13
监管               0      0      0  17    3  0  2    0   22
行业               2      2      0   5    4  1  0    0   14
资本市场             0      0      0  25    0  0  0    0   25
All             20     10      7  70    8  3  8   13  139

# 保存本文件

In [ ]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)